Importing libraries

In [0]:
# Import pytorch basic functions/classes
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Import torchvision functions/classes for MNIST import and data loaders
import torchvision
import torchvision.transforms as transforms

# Set device on which code is run
device = 'cuda'

Defining support functions

In [0]:
# Define support function used to convert label to one-hot encoded tensor
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

Define our network model (the hidden layers size is specified through the constructor)

In [0]:
# Define MLP model and its layers
class Model(nn.Module):

    def __init__(self, hidden_size=1200, dropout=0.0, hidden_dropout=0.0):
        super(Model, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden1 = nn.Linear(784, hidden_size, bias=True)
        self.hidden1_dropout = nn.Dropout(hidden_dropout)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.hidden2_dropout = nn.Dropout(hidden_dropout)
        self.hidden3 = nn.Linear(hidden_size, 10, bias=True)

    def forward(self, x):
        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.hidden1_dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.hidden2_dropout(x)
        x = self.hidden3(x)
        return x#, F.softmax(x)

Downloading MNIST dataset

In [5]:
# Define transform from PIL image to tensor and normalize to 1x768 pixels
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

# Set batch size for data loaders
batch_size = 128

# (Down)load training set
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# (Down)load test set
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Training the Deep Teacher Neural Network

In [17]:
# Setup model and move it to the GPU
net = Model(dropout=0.2, hidden_dropout=0.5)
net.to(device)

# Set up loss function and optimizer: 
#     using cross entropy loss because it's better for classification task

criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.00001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100): #200 for SGD reaches 0.002 loss

    running_loss = 0.0
    total = 0
    for i, data in enumerate(trainloader, 0):
        

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)

        # This for not cross entropy
        #target = convert_labels(labels).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        target = labels.to(device).long()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
PATH = './mnist_dropout_100_epoch.pth'
torch.save(net.state_dict(), PATH)

print('Saved Model')

[1] loss: 0.348
[2] loss: 0.179
[3] loss: 0.141
[4] loss: 0.118
[5] loss: 0.103
[6] loss: 0.092
[7] loss: 0.083
[8] loss: 0.078
[9] loss: 0.071
[10] loss: 0.067
[11] loss: 0.064
[12] loss: 0.061
[13] loss: 0.057
[14] loss: 0.056
[15] loss: 0.052
[16] loss: 0.051
[17] loss: 0.049
[18] loss: 0.046
[19] loss: 0.045
[20] loss: 0.043
[21] loss: 0.042
[22] loss: 0.042
[23] loss: 0.040
[24] loss: 0.039
[25] loss: 0.037
[26] loss: 0.036
[27] loss: 0.036
[28] loss: 0.036
[29] loss: 0.034
[30] loss: 0.033
[31] loss: 0.032
[32] loss: 0.031
[33] loss: 0.031
[34] loss: 0.029
[35] loss: 0.030
[36] loss: 0.029
[37] loss: 0.029
[38] loss: 0.028
[39] loss: 0.027
[40] loss: 0.027
[41] loss: 0.027
[42] loss: 0.026
[43] loss: 0.026
[44] loss: 0.025
[45] loss: 0.025
[46] loss: 0.024
[47] loss: 0.025
[48] loss: 0.023
[49] loss: 0.023
[50] loss: 0.024
[51] loss: 0.023
[52] loss: 0.022
[53] loss: 0.022
[54] loss: 0.022
[55] loss: 0.022
[56] loss: 0.021
[57] loss: 0.021
[58] loss: 0.021
[59] loss: 0.020
[60] l

Run Deep Teacher model on test set

In [18]:
# Instantiate model and load saved network parameters
net = Model().to(device)
net.load_state_dict(torch.load(PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on the 10000 test images: 98 % (132 wrong out of 10000)


Train student model to mimic the teacher

In [19]:
# Custom student loss: linear combination of 2 MSE losses
#     The first one between student output and hard labels
#     The second one between student output and soft labels from teacher

def student_loss(outputA, targetA, outputB, targetB, weight):

    loss = torch.mean((weight*(outputA - targetA)**2) + (1-weight)*(outputB-targetB)**2)

    return loss

# Setup student model and move it to the GPU
student_net = Model(hidden_size = 800)
student_net.to(device)

# Set up loss function and optimizer

#optimizer = optim.SGD(student_net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(student_net.parameters(), lr=0.0001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100):
    running_loss = 0.0
    total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # Set temperature and the weights for losses linear combination
        w = 0.3
        T = 20

        # Compute soft labels using deep teacher model previously trained
        outputs_teacher = net(inputs)
        soft_labels = F.softmax(outputs_teacher/T, dim = 1)

        # Student forward + backward + optimize
        outputs_stud = student_net(inputs)
        loss = student_loss(outputs_stud, target, outputs_stud, soft_labels, w)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
STUD_PATH = './mnist_student_100_epoch.pth'
torch.save(student_net.state_dict(), STUD_PATH)

print('Saved Model')

[1] loss: 0.008
[2] loss: 0.007
[3] loss: 0.007
[4] loss: 0.007
[5] loss: 0.007
[6] loss: 0.007
[7] loss: 0.007
[8] loss: 0.007
[9] loss: 0.007
[10] loss: 0.007
[11] loss: 0.007
[12] loss: 0.007
[13] loss: 0.007
[14] loss: 0.007
[15] loss: 0.007
[16] loss: 0.007
[17] loss: 0.007
[18] loss: 0.007
[19] loss: 0.007
[20] loss: 0.007
[21] loss: 0.007
[22] loss: 0.007
[23] loss: 0.007
[24] loss: 0.007
[25] loss: 0.007
[26] loss: 0.007
[27] loss: 0.007
[28] loss: 0.006
[29] loss: 0.006
[30] loss: 0.006
[31] loss: 0.006
[32] loss: 0.006
[33] loss: 0.006
[34] loss: 0.006
[35] loss: 0.006
[36] loss: 0.006
[37] loss: 0.006
[38] loss: 0.006
[39] loss: 0.006
[40] loss: 0.006
[41] loss: 0.006
[42] loss: 0.006
[43] loss: 0.006
[44] loss: 0.006
[45] loss: 0.006
[46] loss: 0.006
[47] loss: 0.006
[48] loss: 0.006
[49] loss: 0.006
[50] loss: 0.006
[51] loss: 0.006
[52] loss: 0.006
[53] loss: 0.006
[54] loss: 0.006
[55] loss: 0.006
[56] loss: 0.006
[57] loss: 0.006
[58] loss: 0.006
[59] loss: 0.006
[60] l

Running student model on test set

In [20]:
stud_net = Model(hidden_size = 800).to(device)
stud_net.load_state_dict(torch.load(STUD_PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = stud_net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on the 10000 test images: 98 % (140 wrong out of 10000)
